# 공공 데이터 가져오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
# 만개의 레시피 공공 데이터 불러오기
recipe = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/크롤링/recipe.csv', encoding='cp949')

In [4]:
# 인덱스 설정
recipe.set_index('id', inplace = True)

In [5]:
recipe.sample()

,name,composition,ingredients,servings,difficulty
id,,,,,
6901293,불고기우동샐러드,가볍게,[필수재료] 불고기용 소고기 200g| 우동면 1개| 양상추 2줌 [고기양념재료] ...,2인분,초급


In [6]:
# 레시피 id 리스트
id_list = recipe.index.to_list()
print(len(id_list))

3704


# 공공 데이터에 크롤링 정보 추가하기

In [7]:
!pip install BeautifulSoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import requests
from bs4 import BeautifulSoup
import re

In [9]:
# 레시피 id로 레시피 추가 정보 크롤링

recipe_thumbnail = [] # 레시피 썸네일
recipe_time = [] # 레시피 시간
recipe_orders = [] # 레시피 조리 순서
recipe_photo = [] # 레시피 사진
    
def RecipeCrawler(recipeId):
    
    url = f'https://www.10000recipe.com/recipe/{recipeId}'

    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # 레시피 썸네일
    try:
        thumbnail = soup.select_one('#contents_area > div.view2_pic > div.centeredcrop img')['src']
        recipe_thumbnail.append(thumbnail)
    except AttributeError:
        recipe_thumbnail.append('')
    pass

    # 레시피 시간
    try:
        time = soup.select_one('#contents_area > div.view2_summary.st3 > div.view2_summary_info > span.view2_summary_info2').get_text()
        recipe_time.append(time)
    except AttributeError:
        recipe_time.append('')
    pass

    # 레시피 조리 순서
    try:
        elements = soup.find_all('div', 'view_step_cont')
        tmp = ''
        for i, element in enumerate(elements, start=1):
            order = element.find('div', 'media-body')
            order = ''.join(order.findAll(string=True, recursive=False)).strip() # 조리 순서에서 설명, 주방 도구 제외
            tmp = tmp + ('|' + str(i) + '. ' + order)
        recipe_orders.append(tmp)
    except AttributeError:
        recipe_orders.append('')
    pass

    # 레시피 조리 사진
    try:
        elements = soup.find_all('div', 'view_step_cont')
        tmp = ''
        for i, element in enumerate(elements, start=1):
            try:
                photo = element.find('img')['src']
                tmp = tmp + ('|' + photo)
            except TypeError:
                tmp = tmp + ('|' + ' ')
            pass
        recipe_photo.append(tmp)
    except AttributeError:
        recipe_photo.append('')
    pass

In [10]:
# 10개만 테스트
for i, id in enumerate(id_list[:10], start=1):
    print(str(i) + "번째 크롤링 id: " + str(id))
    RecipeCrawler(id)

1번째 크롤링 id: 6842324
2번째 크롤링 id: 6845721
3번째 크롤링 id: 6845906
4번째 크롤링 id: 6846020
5번째 크롤링 id: 6846262
6번째 크롤링 id: 6846342
7번째 크롤링 id: 6846428
8번째 크롤링 id: 6846512
9번째 크롤링 id: 6846633
10번째 크롤링 id: 6846770


In [11]:
# 10개만 테스트
curry_recipe = recipe[:10]

In [12]:
# 추가 정보 열 추가
curry_recipe['thumbanil'] = recipe_thumbnail
curry_recipe['time'] = recipe_time
curry_recipe['orders'] = recipe_orders
curry_recipe['photo'] = recipe_photo

<ipython-input-12-78a51626ca36>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curry_recipe['thumbanil'] = recipe_thumbnail
<ipython-input-12-78a51626ca36>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curry_recipe['time'] = recipe_time
<ipython-input-12-78a51626ca36>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [13]:
curry_recipe.sample()

,name,composition,ingredients,servings,difficulty,thumbanil,time,orders,photo
id,,,,,,,,,
6846633,달걀빵,가볍게,[필수재료] 모닝빵 4개| 소금 약간| 후춧가루 약간 [토핑] 다진베이컨| 햄 4큰...,2인분,아무나,https://recipe1.ezmember.co.kr/cache/recipe/20...,10분 이내,|1. 모닝빵은 윗면에 열십자(+)로 깊게 칼집을 넣는다.|2. 전자레인지용 그릇에...,|https://recipe1.ezmember.co.kr/cache/recipe/2...


# CSV 파일 내보내기

In [14]:
# csv 파일 내보내기
curry_recipe.to_csv('curry_recipe.csv', encoding='utf-8-sig')